In [1]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_enrich_recipes.json') as f:
    gpt4_enrich_recipe = json.load(f)

In [2]:
import sys
sys.path.append('/Users/divyadhara/Documents/CPSC597/ai-meal-recommendation')
from utils.models import EnrichRecipesWithEnergy,IngredientsEnrich, IngredientWithEngergy,EnrichRecipeWithEnergy, EnrichRecipes

In [3]:
recipes=EnrichRecipesWithEnergy(**gpt4_enrich_recipe)

In [4]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_enrich_ingredients.json') as f:
    gpt4_enrich_ingredients = json.load(f)

In [5]:
ingredients_enrich = IngredientsEnrich(**gpt4_enrich_ingredients)

In [6]:
def retrieve_enrich_ingredient(ingredients_enrich:IngredientsEnrich, id:int):
    for item in ingredients_enrich.ingredients:
        if id == item.recipe_id:
            return item

In [7]:
from typing import List
def retrieve_ingredient_energy(ingredient_with_energy:List[IngredientWithEngergy], ingredient_name:str):
    for item in ingredient_with_energy:
        if ingredient_name == item.name:
            if item.usda_food_ingredient:
                return item.usda_food_ingredient.energy
    return 'NA'

In [8]:
def calculate_ingredient_actual_energy(weight, energy):
    if weight == 'NA' or energy == 'NA':
        return 'NA'
    actual_energy = (float(energy)/100.0) * float(weight)
    return round(actual_energy, 2)
        

In [9]:
from pydantic import BaseModel
from typing import List, Optional
class RecipeWithEnergy(BaseModel):
    id: str
    name: str
    usda_calorie_estimation: float| str

class RecipesWithEnergy(BaseModel):
    recipe_with_energy: List[RecipeWithEnergy] = []

In [15]:
def calculate_total_energy_for_recipe(id:str,recipe:EnrichRecipeWithEnergy):
    ingredient = retrieve_enrich_ingredient(ingredients_enrich, int(id))
    recipe_total_calories = 0
    for ingredient_unit_portion in ingredient.ingredient_unit_portion:
        weight = ingredient_unit_portion.weight
        energy = retrieve_ingredient_energy(recipe.ingredients,ingredient_unit_portion.name)
        actual_energy = calculate_ingredient_actual_energy(weight, energy)
        print(ingredient_unit_portion.name,weight,energy,actual_energy)
        recipe_total_calories +=actual_energy
    recipe_with_energy = RecipeWithEnergy(id=recipe.id, name=recipe.recipe_name,usda_calorie_estimation=round(recipe_total_calories,2))
    return recipe_with_energy

In [10]:
def get_recipe(id:str):
    for recipe in recipes.recipes:
        if id == recipe.id:
            return recipe


In [19]:
recipe = get_recipe('11')

In [20]:
calculate_total_energy_for_recipe('11', recipe)

egg whites 264.0 55.0 145.2
fresh spinach 75.0 26.6 19.95
cremini mushrooms (sliced) 140.0 39.8 55.72
cherry tomatoes (halved) 244.0 22.0 53.68
lowfat feta cheese (crumbled) 75.0 250.0 187.5
olive oil 14.0 361.0 50.54
garlic (minced) 6.0 143.0 8.58
salt 1.5 717.0 10.75
black pepper 0.725 22.9 0.17
fresh basil (chopped, for garnish) 5.4 23.0 1.24


RecipeWithEnergy(id='11', name='Spinach and Mushroom Egg White Frittata', usda_calorie_estimation=533.33)

In [38]:
recipes_with_energy = RecipesWithEnergy()
for recipe in recipes.recipes:
    ingredient = retrieve_enrich_ingredient(ingredients_enrich, int(recipe.id))
    recipe_total_calories = 0
    for ingredient_unit_portion in ingredient.ingredient_unit_portion:
        weight = ingredient_unit_portion.weight
        energy = retrieve_ingredient_energy(recipe.ingredients,ingredient_unit_portion.name)
        actual_energy = calculate_ingredient_actual_energy(weight, energy)
        recipe_total_calories +=actual_energy
    recipe_with_energy = RecipeWithEnergy(id=recipe.id, name=recipe.recipe_name,usda_calorie_estimation=round(recipe_total_calories,2))
    recipes_with_energy.recipe_with_energy.append(recipe_with_energy)

In [41]:
recipes_with_energy

RecipesWithEnergy(recipe_with_energy=[RecipeWithEnergy(id='14', name='Mediterranean Chickpea Salad', usda_calorie_estimation=1921.6), RecipeWithEnergy(id='15', name='Mediterranean Stuffed Mushrooms', usda_calorie_estimation=503.92), RecipeWithEnergy(id='17', name='Tangy Greek Yogurt Dressing', usda_calorie_estimation=480.66), RecipeWithEnergy(id='16', name='Hearty Vegetable Lentil Soup', usda_calorie_estimation=1045.53), RecipeWithEnergy(id='12', name='Mediterranean Chickpea Salad', usda_calorie_estimation=2186.99), RecipeWithEnergy(id='13', name='Green Veggie Diabetic-Friendly Smoothie', usda_calorie_estimation=295.84), RecipeWithEnergy(id='11', name='Spinach and Mushroom Egg White Frittata', usda_calorie_estimation=533.33), RecipeWithEnergy(id='20', name='Cucumber Mint Infused Water', usda_calorie_estimation=49.58), RecipeWithEnergy(id='18', name='Zucchini Noodles with Pesto and Cherry Tomatoes', usda_calorie_estimation=918.1), RecipeWithEnergy(id='19', name='Zucchini Noodles with Pe

In [67]:
import re

def extract_ingredient_calories(text):
    """
    Extracts the lines listing ingredients and their calorie values.
    
    Args:
    text (str): Multi-line string from which to extract ingredient calorie information.
    
    Returns:
    str: Extracted lines with ingredient calorie information.
    """
    # Regex pattern to find the section with ingredient calories
    pattern = r"- Calories for each ingredient:(.*?)(?=\n- [A-Za-z]|\Z)"

    
    # Use re.DOTALL to make '.' match newlines as well
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Further extract individual lines with calorie information
        ingredient_lines = match.group(1).strip()
        return ingredient_lines
    else:
        return "No calorie information found."

def calculate_total_calories(ingredients_info):
    """
    Calculate the total calories from a given string of ingredients with their calorie information,
    including handling multipliers and specified servings.

    Args:
        ingredients_info (str): String containing ingredient names followed by complex calorie descriptions.

    Returns:
        int: Total calories calculated from the ingredients.
    """
    total_calories = 0

    # Extract lines and calculate based on expressions
    for line in ingredients_info.split('\n'):
        # Search for patterns like "100 calories each x 10 ="
        match = re.search(r'(\d+)\s+calories.*?(\d+)\s*=', line)
        if match:
            # Calculate total calories for the line by multiplying
            per_unit_calories = int(match.group(1))
            count = int(match.group(2))
            total_calories += per_unit_calories * count
        else:
            # For simple calorie statements
            match = re.search(r'(\d+)\s+calories', line)
            if match:
                total_calories += int(match.group(1))

    return total_calories

# Sample input text
text = """
- Total Calories for entire recipe : Approximately 400 calories for the entire recipe. \n- Calories for each ingredient (using USDA Food API where applicable):\n    Egg Whites: 70 calories (from 8 egg whites)\n    Fresh Spinach: 20 calories\n    Cremini Mushrooms: 30 calories\n    Cherry Tomatoes: 30 calories\n    Low-fat Feta Cheese: 140 calories\n    Olive Oil: 120 calories\n    Garlic: Negligible\n    Salt, Black Pepper: Negligible\n    Fresh Basil: Negligible\n- Serving People: 4
"""

# Extracting the calorie information
extracted_calories = extract_ingredient_calories(text)
print(extracted_calories)
# Calculating total calories
total_calories = calculate_total_calories(extracted_calories)
print(f"Total Calories: {total_calories}")

No calorie information found.
Total Calories: 0


In [44]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_ner_recipes.json') as f:
    gpt4_recipe_ner = json.load(f)

In [49]:
recipes_ner = EnrichRecipes(**gpt4_recipe_ner)

In [68]:
gpt4_turbo_total_calories = []
for item in recipes_ner.recipes:
    temp = {}
    temp['id'] = item.id
    extracted_calories = extract_ingredient_calories(item.total_calories_estimation)
    temp['gpt4_recipe_total_calories']  = calculate_total_calories(extracted_calories)
    gpt4_turbo_total_calories.append(temp)

In [69]:
total_calories_estimation = []
for item in gpt4_turbo_total_calories:
    calories_estimation = {}
    recipe_with_energy=get_recipe_with_energy(item['id'], recipes_with_energy)
    calories_estimation['id'] = recipe_with_energy.id
    calories_estimation['name'] = recipe_with_energy.name
    calories_estimation['usda_calorie_estimation'] = recipe_with_energy.usda_calorie_estimation
    calories_estimation['gpt4_calorie_estimation'] = item['gpt4_recipe_total_calories']
    total_calories_estimation.append(calories_estimation)

In [70]:
total_calories_estimation

[{'id': '14',
  'name': 'Mediterranean Chickpea Salad',
  'usda_calorie_estimation': 1921.6,
  'gpt4_calorie_estimation': 1400},
 {'id': '15',
  'name': 'Mediterranean Stuffed Mushrooms',
  'usda_calorie_estimation': 503.92,
  'gpt4_calorie_estimation': 0},
 {'id': '17',
  'name': 'Tangy Greek Yogurt Dressing',
  'usda_calorie_estimation': 480.66,
  'gpt4_calorie_estimation': 368},
 {'id': '16',
  'name': 'Hearty Vegetable Lentil Soup',
  'usda_calorie_estimation': 1045.53,
  'gpt4_calorie_estimation': 1055},
 {'id': '12',
  'name': 'Mediterranean Chickpea Salad',
  'usda_calorie_estimation': 2186.99,
  'gpt4_calorie_estimation': 1358},
 {'id': '13',
  'name': 'Green Veggie Diabetic-Friendly Smoothie',
  'usda_calorie_estimation': 295.84,
  'gpt4_calorie_estimation': 264},
 {'id': '11',
  'name': 'Spinach and Mushroom Egg White Frittata',
  'usda_calorie_estimation': 533.33,
  'gpt4_calorie_estimation': 0},
 {'id': '20',
  'name': 'Cucumber Mint Infused Water',
  'usda_calorie_estimati